# Grid Search on Model's Parameters


## Objective

Investigate if a reasonable prediction on video popularity could be made in shorter time than suggested by the literature.

## Description

Different setups of early and late popularity are run.

The models produced are evaluated according to cross-validation(k=4) on two metrics:
- r2 score (best possible value is 1.0)
- negative relative square error (best possible value is 0.0)

The mean score and a 95% trust margin are shown for every model evaluated.

Youtube and Facebook data are evaluated separatedly. No grouping by meta-tag is performed on this experiment.

In [2]:
# %load main.py
from sklearn import linear_model

from year_ap_predictive import predictive_framework

from data_sources.linear_log_popularity_data_source import EarlyToLatePopularityDataSource
from report_writer import write_report

CONFIG_FILE = '../config.json'


def scores_statistics(scores):
    return (scores.mean(), 2 * scores.std())


data_source = EarlyToLatePopularityDataSource(7, 30, CONFIG_FILE)
model = linear_model.LinearRegression

data_source.open_connection()

early_ages = [1, 2, 3, 4, 5, 6, 7]
late_ages = [1, 2, 3, 4, 5, 6, 7, 30]
experiments = []
for early in early_ages:
    for late in late_ages:
        if early >= late:
            continue
        data_source.set_params(early, late)
        (group_data, group_models, group_scores) = predictive_framework.run(data_source, model)

        for group_key, v in group_scores.items():
            r2_score = scores_statistics(v['test_r2'])
            rms_score = scores_statistics(v['test_rse'])
            experiments.append((early, late, group_key, r2_score[0], r2_score[1], rms_score[0], rms_score[1]))

data_source.close_connection()
write_report(experiments)


## facebook
### R2 Error
    2             3             4             5             6             7             30
--  ------------  ------------  ------------  ------------  ------------  ------------  ------------
 1  0.95 +- 0.10  0.59 +- 1.23  0.57 +- 1.22  0.57 +- 1.21  0.56 +- 1.21  0.53 +- 1.27  0.63 +- 1.03
 2  -             0.99 +- 0.00  0.97 +- 0.02  0.95 +- 0.01  0.94 +- 0.02  0.91 +- 0.05  0.78 +- 0.14
 3  -             -             0.99 +- 0.01  0.98 +- 0.02  0.96 +- 0.03  0.94 +- 0.04  0.81 +- 0.11
 4  -             -             -             0.99 +- 0.02  0.97 +- 0.04  0.95 +- 0.05  0.81 +- 0.11
 5  -             -             -             -             0.99 +- 0.02  0.97 +- 0.04  0.85 +- 0.13
 6  -             -             -             -             -             0.99 +- 0.04  0.87 +- 0.15
 7  -             -             -             -             -             -             0.91 +- 0.06
### Negative Relative Square Error
    2              3              4      

## Conclusion

The setup (7, 30) proposed by the literature scored r2 0.91 on facebook, and 0.93 on youtube. A much earlier prediction with setup (3, 5) scored 0.98 on facebbok and 0.95 on youtube - a better result on both platforms. According to our view, this experiment suggests that earlier predictions based on the proposed model are indeed possible.

Another finding of this experiment was that rse doesn't seem like a fit metric for this model, as it produced estimates of the quality of the model that deviate a lot depending on the particular selection of training and test data.

## Future Work

The fact that rse produced very high standard error on score estimation is intriging. In my opinion, this suggests that cross validation with k=4 might not be suited to compute a 95% trust margin on the score estimation. On future development I'd suggest running repeated cross-validation with a higher partitioning and re-evaluation of the error margin. The goal would be to produce a more precise rse score estimation. The error estimate should also be run considering grouping by meta tag.